In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
import os
import re
import time

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import AUC

from kaggle_datasets import KaggleDatasets

import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

EPOCHS = 20
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 200
MODEL = 'roberta-base'

In [ ]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [ ]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [ ]:
# def build_model(transformer, max_len=512):

#     input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
#     sequence_output = transformer(input_word_ids)[0]
#     cls_token = sequence_output[:, 0, :]
#     out = Dense(3, activation='softmax')(cls_token)
    
#     model = Model(inputs=input_word_ids, outputs=out)
#     model.compile(Adam(lr=1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
#     return model

In [ ]:
def create_model_and_optimizer():
    with strategy.scope():
        transformer_layer = TFAutoModel.from_pretrained(MODEL)                
        model = build_model(transformer_layer)
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, epsilon=1e-08)
    return model, optimizer


def build_model(transformer):
    inp = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")
    x = transformer(inp)[0][:, 0, :]  
    out = Dense(3, activation='softmax')(x)
    model = Model(inputs=[inp], outputs=[out])
    return model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
train = pd.read_csv('../input/60k-stack-overflow-questions-with-quality-rate/train.csv')

valid = pd.read_csv('../input/60k-stack-overflow-questions-with-quality-rate/valid.csv')

train.shape, valid.shape

In [ ]:
train['text'] = train['Title'] + " " + train['Body']

train.drop(['Id', 'Tags', 'CreationDate', 'Title', 'Body'], axis=1, inplace=True)

train = train.rename(columns={"Y": "class"})

print("Total number of samples:", len(train))

train.head()

In [ ]:
valid['text'] = valid['Title'] + " " + valid['Body']

valid.drop(['Id', 'Tags', 'CreationDate', 'Title', 'Body'], axis=1, inplace=True)

valid = valid.rename(columns={"Y": "class"})

print("Total number of samples:", len(valid))

valid.head()

In [ ]:
train['text'] = train['text'].str.replace('<(.|\n)*?>', ' ')
train['text'] = train['text'].str.lower()
train['text'] = train['text'].str.replace('[^(a-zA-Z)\s]', '')

valid['text'] = valid['text'].str.replace('<(.|\n)*?>', ' ')
valid['text'] = valid['text'].str.lower()
valid['text'] = valid['text'].str.replace('[^(a-zA-Z)\s]', '')

train.head()

In [ ]:
%%time

x_train = regular_encode(train['text'].values, tokenizer, maxlen=MAX_LEN)

x_valid = regular_encode(valid['text'].values, tokenizer, maxlen=MAX_LEN)

In [ ]:
y_train = train['class'].replace(['HQ', 'LQ_EDIT', 'LQ_CLOSE'], [2, 1, 0])

y_valid = valid['class'].replace(['HQ', 'LQ_EDIT', 'LQ_CLOSE'], [2, 1, 0])

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

In [ ]:
%%time

model, optimizer = create_model_and_optimizer()
model.summary()

In [ ]:
filepath = "roberta_model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='max')

early = EarlyStopping(monitor='val_accuracy', mode='max', patience=5)

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=2, min_lr=1e-5)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
n_steps = x_train.shape[0] // BATCH_SIZE

train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS,
    callbacks=[early, checkpoint, reduce_lr]
)

In [ ]:
model.load_weights('roberta_model.h5')

In [ ]:
model_accuracy = model.evaluate(x_valid, y_valid)[1]

print('Model Accuracy: '+ str(round(model_accuracy*100, 2))+'%')